In [6]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, BatchNormalization, ReLU, Dropout
from tensorflow.keras.models import Model, clone_model
from itertools import product

import functions as fs
from functions import make_state, run, get_optimal_value, get_optimal_action, get_optimal_actions, get_model_actions, test_model, one_batch, test_model_accuracy, one_batch_supervised, train, train_supervised, create_model


## 测试初始值

In [9]:
def init_test(n_tests):
    optimilizer = 'adam'
    n_hidden_layers = 1
    n_dense_units = 512
    ratio_dropout = 0.5
    res = []
    for i in np.arange(n_tests):
        model = create_model(n_hidden_layers, n_dense_units, ratio_dropout, optimilizer)
        scores = np.array([one_batch(model, batch_size=128, n_test_rounds=10000) for i in np.arange(200)])
        argmax = scores.argmax()
        res.append((argmax, scores[argmax]))
        print(argmax, scores[argmax])
    return res

In [10]:
res = init_test(100)

91 0.8871047291815229
85 0.8862419076862652
99 0.8975701829041489
100 0.8903939813782483
99 0.8967762625158786
76 0.9001704130775677
104 0.8878177908665682
80 0.8915584423206854
69 0.87864929156894
86 0.8817003765261404
61 0.8992280854481282
70 0.8935994494471703
99 0.8920077490433727
90 0.8804108882423556
86 0.9028971175340171
97 0.8994224696917507
98 0.8843926990021138
120 0.8859382810970584
83 0.8796277577861581
74 0.8878206366660072
76 0.8971136421217442
104 0.8869598028007051
59 0.8771655590854928
70 0.8950496111751934
119 0.8821364803773045
80 0.8771734242681872
72 0.8792567381053886
80 0.901081881471236
95 0.8877519696559931
87 0.893883839918055
96 0.8967838638079609
89 0.9076946365686636
106 0.8879401988618103
98 0.8818786590107557
77 0.8887972458985943
97 0.8864693669851066
73 0.8908471231907359
72 0.8827566540911547
84 0.88507517809997
106 0.8937099700337452
72 0.8993008840343003
93 0.8988467096829621
107 0.8859560970193728
70 0.8844518365191428
94 0.9099911817272972
96 0.879